<a href="https://colab.research.google.com/github/geokos2024/George_Kosmas/blob/main/Lambda_IPS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#Συλλογή Δεδομένων
import os
import pandas as pd
from scapy.all import sniff, wrpcap

def collect_data(output_file='network_traffic.pcap', duration=60):
    """
    Collects network traffic data for a given duration and saves it to a pcap file.

    :param output_file: The file to which the network traffic data will be saved.
    :param duration: The duration for which to collect the network traffic data (in seconds).
    """
    print(f"Collecting data for {duration} seconds...")
    packets = sniff(timeout=duration)
    wrpcap(output_file, packets)
    print(f"Data collected and saved to {output_file}")

# Example usage
collect_data(duration=60)

ModuleNotFoundError: No module named 'scapy'

In [10]:
#Επεξεργασία Δεδομένων
from scapy.all import rdpcap

def process_data(input_file='network_traffic.pcap'):
    """
    Processes the collected network traffic data to extract features.

    :param input_file: The pcap file containing the collected network traffic data.
    :return: A DataFrame with extracted features.
    """
    packets = rdpcap(input_file)
    features = []

    for pkt in packets:
        feature = {
            'src_ip': pkt[1].src if 1 in pkt else None,
            'dst_ip': pkt[1].dst if 1 in pkt else None,
            'src_port': pkt[2].sport if 2 in pkt else None,
            'dst_port': pkt[2].dport if 2 in pkt else None,
            'protocol': pkt[2].name if 2 in pkt else None,
            'length': len(pkt)
        }
        features.append(feature)

    df = pd.DataFrame(features)
    df = df.dropna()  # Remove rows with missing values
    return df

# Example usage
df = process_data()
print(df.head())

ModuleNotFoundError: No module named 'scapy'

In [11]:
#Χρήση Τεχνικών Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

def analyze_data(df):
    """
    Analyzes the processed data using machine learning to detect malicious patterns.

    :param df: DataFrame with extracted features.
    :return: Trained machine learning model.
    """
    # For demonstration purposes, we'll assume a simple labeling scheme
    df['label'] = (df['protocol'] == 'TCP') & (df['length'] > 1000)  # Simplistic rule for labeling

    X = df[['src_ip', 'dst_ip', 'src_port', 'dst_port', 'protocol', 'length']]
    y = df['label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

    return model

# Example usage
model = analyze_data(df)

KeyError: 'protocol'

In [12]:
#Import Libraries and Load Data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load your processed data
data = pd.read_csv('processed_network_data.csv')  # Assuming you have a CSV file with processed data

# Prepare features and labels
X = data.drop('label', axis=1)
y = data['label']

# Encode categorical variables if necessary
le = LabelEncoder()
X['protocol'] = le.fit_transform(X['protocol'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

FileNotFoundError: [Errno 2] No such file or directory: 'processed_network_data.csv'

In [13]:
#Training with Support Vector Machines (SVM)
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Train the SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Make predictions
y_pred_svm = svm_model.predict(X_test)

# Evaluate the model
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))

NameError: name 'X_train' is not defined

In [ ]:
#Training with Random Forests
from sklearn.ensemble import RandomForestClassifier

# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))


In [ ]:
#Training with Neural Networks
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the neural network model
nn_model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred_nn = (nn_model.predict(X_test) > 0.5).astype("int32")

print("Neural Network Classification Report:")
print(classification_report(y_test, y_pred_nn))

In [ ]:
#Αποκλεισμός Υπόπτου Κυκλοφορίας
from scapy.all import sniff, IP

# Define a function to detect and block suspicious packets
def packet_filter(packet):
    if IP in packet:
        # Example criterion: Block packets from a specific IP address
        if packet[IP].src == '192.168.1.100':
            print(f"Blocking packet from {packet[IP].src}")
            return False  # Drop the packet
    return True  # Allow the packet

# Sniff the network and apply the packet filter
sniff(filter="ip", prn=packet_filter, store=0)


In [ ]:
#Απομόνωση Συσκευών
import subprocess

# Define a function to isolate a device by blocking its MAC address
def isolate_device(mac_address):
    # Example command to add a MAC address to a block list (depends on router/switch capabilities)
    command = f"sudo iptables -A INPUT -m mac --mac-source {mac_address} -j DROP"
    subprocess.run(command, shell=True)
    print(f"Isolated device with MAC address: {mac_address}")

# Isolate a specific device
isolate_device('00:14:22:01:23:45')


In [ ]:
#Ειδοποίηση Διαχειριστών
import smtplib
from email.mime.text import MIMEText

# Define a function to send an email notification
def send_alert(subject, body, to_email):
    from_email = 'your_email@example.com'
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email

    # Send the email
    with smtplib.SMTP('smtp.example.com') as server:
        server.login('your_email@example.com', 'your_password')
        server.sendmail(from_email, [to_email], msg.as_string())
    print(f"Alert sent to {to_email}")

# Send an alert
send_alert('Security Alert', 'Suspicious activity detected.', 'admin@example.com')

In [ ]:
#Φόρτωση του Εκπαιδευμένου Μοντέλου
import joblib

# Load the trained machine learning model
model = joblib.load('intrusion_detection_model.pkl')

In [ ]:
#Ορισμός Κανόνων
def apply_rules(prediction, packet):
    if prediction == 1:  # Assuming 1 indicates a threat
        # Block suspicious traffic
        block_packet(packet)
        # Isolate the device
        isolate_device(packet.src)
        # Send an alert to administrators
        send_alert('Security Alert', 'Suspicious activity detected.', 'admin@example.com')

def block_packet(packet):
    print(f"Blocking packet from {packet.src}")

def isolate_device(mac_address):
    command = f"sudo iptables -A INPUT -m mac --mac-source {mac_address} -j DROP"
    subprocess.run(command, shell=True)
    print(f"Isolated device with MAC address: {mac_address}")

def send_alert(subject, body, to_email):
    from_email = 'your_email@example.com'
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email

    with smtplib.SMTP('smtp.example.com') as server:
        server.login('your_email@example.com', 'your_password')
        server.sendmail(from_email, [to_email], msg.as_string())
    print(f"Alert sent to {to_email}")

In [ ]:
#Ενσωμάτωση στο Σύστημα IPS
from scapy.all import sniff, IP

# Define a function to detect and respond to threats
def detect_and_respond(packet):
    if IP in packet:
        features = extract_features(packet)
        prediction = model.predict([features])[0]
        apply_rules(prediction, packet)

def extract_features(packet):
    # Extract relevant features from the packet for the model
    features = [
        packet[IP].len,
        packet[IP].ttl,
        # Add more features as needed
    ]
    return features

# Sniff the network and apply the detection and response function
sniff(filter="ip", prn=detect_and_respond, store=0)

In [ ]:
#Συλλογή Νέων Δεδομένων
import pandas as pd

def collect_new_data():
    # Replace with actual data collection logic
    new_data = pd.read_csv('new_network_data.csv')
    return new_data


In [ ]:
#Προεπεξεργασία Νέων Δεδομένων
from sklearn.preprocessing import StandardScaler

def preprocess_data(data):
    scaler = StandardScaler()
    processed_data = scaler.fit_transform(data)
    return processed_data

In [ ]:
#Ενημέρωση του Μοντέλου με Νέα Δεδομένα
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

def update_model(new_data, model):
    X = new_data.drop('label', axis=1)
    y = new_data['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train, y_train)
    joblib.dump(model, 'updated_intrusion_detection_model.pkl')
    return model

In [ ]:
#Επαναξιολόγηση των Κανόνων
def revaluate_rules(model, new_data):
    predictions = model.predict(new_data.drop('label', axis=1))
    for prediction, data in zip(predictions, new_data.iterrows()):
        apply_rules(prediction, data[1])

In [ ]:
#Συνεχής Παρακολούθηση και Αναβάθμιση
import schedule
import time

def job():
    new_data = collect_new_data()
    processed_data = preprocess_data(new_data)
    model = joblib.load('intrusion_detection_model.pkl')
    updated_model = update_model(processed_data, model)
    revaluate_rules(updated_model, new_data)

# Schedule the job to run daily
schedule.every().day.at("00:00").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
#τελική αξιολόγηση του μοντέλου στο σύνολο δοκιμών
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
blocked_indices = np.where(y_pred_proba > blocking_threshold)[0]
if len(blocked_indices) > 0:
    print(f"Blocking {len(blocked_indices)} malicious traffic instances.")

accuracy = accuracy_score(y_test, y_pred_proba > blocking_threshold)
classification_rep = classification_report(y_test, y_pred_proba > blocking_threshold)

#εκτύπωση τελικών αποτελεσμάτων
print(f"Final Test Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)